In [2]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [8]:
df = pd.read_excel('40_mfcc.xlsx')
df = df.sample(frac=1).reset_index(drop=True)
df1 = df.drop(['Varience of Energy','Varience of Pitch'],axis=1)
df1.columns = df1.columns.astype(str)
df1 = df1.round(2)
df1.shape

(2800, 45)

In [19]:
from scipy.stats import zscore
data_z = df1.copy()
z_scores = data_z.apply(zscore)

# Filter out rows where any Z-score is greater than 3
df_filtered = data_z[(z_scores.abs()<=3).all(axis=1)]
df_filtered.shape

(2140, 45)

In [20]:
df_delete = data_z[(z_scores.abs() > 3).any(axis=1)]
df_delete.shape

(660, 45)

In [21]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
df_filtered['LOF_Score'] = lof.fit_predict(df_filtered)

# Outliers are marked as -1
outliers = df_filtered[df_filtered['LOF_Score'] == -1]

print("Rows with outliers using LOF:")
print(outliers.shape)
df_local_filtered = df_filtered[df_filtered.LOF_Score == 1]

Rows with outliers using LOF:
(107, 46)


C:\Users\Dell\AppData\Local\Temp\ipykernel_32184\2132960500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['LOF_Score'] = lof.fit_predict(df_filtered)


In [23]:
print(df_local_filtered.shape)
df2 = df_local_filtered.to_numpy()
x = df2[:,:-2]
y = df2[:,-2]
print('categorical type: ',len(np.unique(y)))

(2033, 46)
categorical type:  7


In [24]:
from tensorflow.keras.utils import to_categorical

# Assuming your labels are in a variable called `y`
y_categorical = to_categorical(y, num_classes=7)
y_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [25]:
X_reshaped = x.reshape(x.shape[0], 1, x.shape[1]) 
X_reshaped

array([[[-3.3907e+02,  2.2570e+01, -7.9100e+00, ...,  6.9900e+00,
         -4.9000e+00,  2.1000e-01]],

       [[-3.2157e+02,  2.4100e+01, -2.2910e+01, ...,  2.5500e+00,
         -9.4100e+00,  2.2000e-01]],

       [[-4.3713e+02,  8.7190e+01,  1.1740e+01, ...,  1.1000e+00,
          1.1930e+01,  1.3000e-01]],

       ...,

       [[-5.5075e+02,  6.5170e+01,  2.4330e+01, ...,  2.5000e-01,
          1.3830e+01,  6.0000e-02]],

       [[-3.2552e+02,  1.5980e+01, -5.0000e+00, ...,  9.6700e+00,
          2.6000e+00,  2.1000e-01]],

       [[-4.0702e+02,  6.2070e+01,  2.8650e+01, ...,  2.3100e+00,
          8.3500e+00,  1.8000e-01]]])

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

model = Sequential()
# Adding an LSTM layer
model.add(LSTM(128, input_shape=(X_reshaped.shape[1], X_reshaped.shape[2]),kernel_regularizer=l2(0.001),return_sequences=False))
model.add(BatchNormalization())
# Adding a dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Adding a dense layer for classification
model.add(Dense(128, activation='relu',kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
# Output layer
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        88,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 106,503 (416.03 KB)

 Trainable params: 106,247 (415.03 KB)

 Non-trainable params: 256 (1.00 KB)

In [27]:
# Train the model
model.fit(X_reshaped, y_categorical, epochs=50, batch_size=64, validation_split=0.25)


Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2591 - loss: 2.0532 - val_accuracy: 0.3713 - val_loss: 1.8140
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4956 - loss: 1.4900 - val_accuracy: 0.6326 - val_loss: 1.5352
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6607 - loss: 1.0806 - val_accuracy: 0.8251 - val_loss: 1.2120
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8503 - loss: 0.6608 - val_accuracy: 0.9214 - val_loss: 0.9056
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9171 - loss: 0.4191 - val_accuracy: 0.9607 - val_loss: 0.7148
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9482 - loss: 0.3064 - val_accuracy: 0.9273 - val_loss: 0.6220
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9580 - loss: 0.2384 - val_accuracy: 0.9804 - val_loss: 0.3498
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9727 - loss: 0.2026 - val_accuracy: 0.9587 - val_loss

In [28]:
model.save('audio_new_lstm_model.keras')